In [10]:
from PIL import Image
import os

def format_image(image):
    r

def load_data(train_ratio = 0.8, image_dir = '/home/jupyter/SSA Project/Generator/output'):    
    labels = []
    
    x = []
    y = []
    
    for filename in os.listdir(image_dir):
        if filename.endswith('.JPEG'):
            with Image.open(os.path.join(image_dir, filename), 'r') as img:
                label_name = ''.join(filename.split('_')[:-1])
                if label_name not in labels:
                    labels.append(label_name)
                y.append(labels.indexOf(label_name))
                x.append(format_image(img))
                    
    
    return (x_train, y_train), (x_test, y_test)

load_data()

AttributeError: 'list' object has no attribute 'indexOf'

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# from lib import datasets
from tensorflow.keras.datasets import cifar10 as datasets

Using TensorFlow backend.


## Grab the JPEGs

In [4]:
image_dir = '/home/jupyter/SSA Project/theDarkPast/Generator/output'
class_folders = [x[0] for x in os.walk(image_dir)][1:-1]
class_folders

['/home/jupyter/SSA Project/theDarkPast/Generator/output/ACRIMSAT',
 '/home/jupyter/SSA Project/theDarkPast/Generator/output/Deep Space',
 '/home/jupyter/SSA Project/theDarkPast/Generator/output/Aqua',
 '/home/jupyter/SSA Project/theDarkPast/Generator/output/Aqua/.ipynb_checkpoints']

In [5]:
ex_samples = os.listdir(class_folders[0])
ex_samples[:5], len(ex_samples)

(['ACRIMSAT_05_47.JPEG',
  'ACRIMSAT_05_79.JPEG',
  'ACRIMSAT_05_15.JPEG',
  'ACRIMSAT_05_32.JPEG',
  'ACRIMSAT_05_65.JPEG'],
 100)

In [8]:
training_num = int(len(ex_samples)*len(class_folders)*.8)
testing_num = int(len(ex_samples)*len(class_folders)*.2)
x_train = np.zeros((training_num, 224,224,3))
x_test = np.zeros((testing_num, 224,224,3))
y_test = np.zeros((testing_num, 1))

x_train.shape, x_test.shape, y_test.shape

((320, 224, 224, 3), (80, 224, 224, 3), (80, 1))

In [40]:
x_train[0].shape

(224, 224, 3)

In [51]:
class_folders[0] + '/' + ex_samples[0]

'/home/jupyter/SSA Project/Generator/output/ACRIMSAT/ACRIMSAT_05_47.JPEG'

In [53]:
img = cv2.imread(class_folders[0] + '/' + ex_samples[0])
img.shape

(224, 224, 3)

In [66]:
train_idx = 0
test_idx = 0

for class_ in class_folders:
    samples = os.listdir(class_)
    x_train_samples = samples[:training_num//len(class_folders)]
    x_test_samples = samples[training_num//len(class_folders):]
    
    for sample in enumerate(x_train_samples):
        x_train[train_idx] = cv2.imread(class_ + '/' + sample[-1])
        train_idx += 1
    
    for sample in x_test_samples:
        x_test[test_idx] = cv2.imread(class_ + '/' + sample[-1])
        test_idx +=1 

In [65]:
sample

(0, 'ACRIMSAT_05_47.JPEG')

In [9]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'satellite_classification_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = datasets.load_data()
num_classes = len(np.unique(y_train))

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(num_classes, 'classes')

NameError: name 'os' is not defined

In [8]:
# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

NameError: name 'y_train' is not defined

In [70]:
model = tf.keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', classes=1000)
model.summary()

102973440/102967424 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [21]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [22]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [23]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 204s 131ms/step - loss: 1.8744 - accuracy: 0.3085 - val_loss: 1.5658 - val_accuracy: 0.4244
Epoch 2/100
1563/1563 [==============================] - 195s 124ms/step - loss: 1.5836 - accuracy: 0.4242 - val_loss: 1.3721 - val_accuracy: 0.5029
Epoch 3/100
1563/1563 [==============================] - 199s 127ms/step - loss: 1.4632 - accuracy: 0.4693 - val_loss: 1.2876 - val_accuracy: 0.5347
Epoch 4/100
1563/1563 [==============================] - 194s 124ms/step - loss: 1.3775 - accuracy: 0.5051 - val_loss: 1.2419 - val_accuracy: 0.5517
Epoch 5/100
 441/1563 [=======>......................] - ETA: 2:14 - loss: 1.3366 - accuracy: 0.5184

KeyboardInterrupt: 

In [1]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

In [1]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using TensorFlow backend.
